# Machine Learning with ToyotaHome/NVIDIA

## READ ME:
Run all cells in ascending order

### Setup/Imports

In [2]:
# Functions to use along widgets
from IPython.display import HTML, display, Markdown, Video, clear_output
# Faciliate file selection
from tkinter import *
from tkinter import filedialog
# Widget Packages
import ipywidgets as widgets
# jupyter nbextension enable --py widgetsnbextension
# Used for local directory
import os
import sys

### Video Selector Functionality

In [3]:
path = os.getcwd() # print(os.path.basename(path)) # get the local directory
videoSelector = widgets.Button(description = "Select Video Preview")
videoClearer = widgets.Button(description = "Clear Video Preview")

#Create the videoplayer here
video_output = widgets.Output() # Customisation

def video_Select(b):
    root = Tk()
    root.filename =  filedialog.askopenfilename(
        initialdir = path,title = "Select file",filetypes = (("videos","*.mp4"),("all files","*.*")))
    FileName = os.path.basename(root.filename)
    # Get directory path
    cwd = os.getcwd()
    # Get file path
    AbsoluteFilePath = os.path.normpath(root.filename)
    # Replace file path's directory nonsense with the relative command
    FilePath = AbsoluteFilePath.replace(cwd,".")
    root.destroy() #Closes the Window
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="100%" height="100%" controls><source src="{filepath}" type="video/mp4"></video>'.format(filepath = FilePath)
        video_output.append_display_data(HTML(html_video_code))

def video_Clear(b):
    with video_output:
        video_output.clear_output()

#Link the button to the function
videoSelector.on_click(video_Select)
videoClearer.on_click(video_Clear)

### Shared Widgets

In [4]:
selected_Directory = widgets.Text(
    description = 'Directory',
    value = '',
    placeholder = 'Select Directory'
    )
current_Directory = widgets.Label()
tooltip_Dir = widgets.Label("Current Directory: ")
selectDir = widgets.Button(description = "Select Directory")
clearDir = widgets.Button(description = "Clear")
confirmDir = widgets.Button(description = "Confirm")

sharedWidgets = widgets.VBox([widgets.HBox([tooltip_Dir,current_Directory]),widgets.HBox([selectDir, confirmDir, clearDir])])

def directory_Select(b):
    root = Tk()
    root.directory =  filedialog.askdirectory (
        initialdir = path)
    selected_Directory.value = root.directory
    root.destroy() #Closes the Window\n",

def directory_Clear(b):
    current_Directory.value = ""

def directory_Print(b):
    current_Directory.value = selected_Directory.value
selectDir.on_click(directory_Select)
clearDir.on_click(directory_Clear)
confirmDir.on_click(directory_Print)

###  Toyota Widgets
These are the widgets required to run Toyota ML

In [5]:
Toyota = widgets.IntSlider(description = "Toyo")

### NVIDIA Widgets
These are the widgets required to run NVIDIA ML

In [6]:
Nvidia = widgets.IntSlider(description = "Nvidia")

In [7]:
# Pre-requisites
"""
List of Machine Learning stuff
Consider making it a text file to store/edit?
"""
ml_list = ['ToyotaHome', 'Nvidia']

# Widgets
mlSelectConfirm = widgets.Button(description="Confirm")
menu = widgets.Dropdown(
    options=ml_list,
    value=ml_list[0],
    description='ML Model:')

# Output
widgetset = widgets.Output()

# Function
def selectWidgetSet(b):
    with widgetset:
        widgetset.clear_output()
        if(menu.value == ml_list[0]): # if Toyota
            display(Toyota, sharedWidgets)
        elif(menu.value == ml_list[1]): # if Nvidia
            display(Nvidia, sharedWidgets)


mlSelectConfirm.on_click(selectWidgetSet)

### Other iPywidgets

#### Relevant links (in case I forget)
Output Widget
https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
<br>Widget List
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
<br>Linking Widgets
https://ipywidgets.readthedocs.io/en/7.6.2/examples/Widget%20Events.html
<br>Clear Output
https://stackoverflow.com/questions/54029719/jupyter-ipywidgets-how-to-clear-cell-output-before-re-printing-the-output
<br>Video Playback
https://towardsdatascience.com/video-streaming-in-the-jupyter-notebook-635bc5809e85

##### Markdown Syntax
https://www.markdownguide.org/basic-syntax

In [8]:
title = widgets.Label("Control Panel")
tooltip = widgets.Label("Supported Video Types: MP4, WebM, and OGG.")

mlBox = widgets.VBox([widgets.HBox([menu, mlSelectConfirm]),widgetset])
videoBox = widgets.VBox([widgets.HBox([videoSelector, videoClearer, tooltip]),video_output])
toolBox = widgets.VBox([title,mlBox,videoBox])

toolBox

### ML Widget Selector Functionality

# 1. Data Exploration

## 1.1 Select a video to playback

In [27]:
# @hidden_cell
## retrieve all videos to dropdown list

def populateList(fileDirectory, fileType):
    folder_files = os.listdir(fileDirectory) #You can also use full path.
    print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
    fileList = []
    for file in folder_files:
        if file.endswith(fileType):
            fileList.append(file)
    return fileList

fileList = populateList('data/rgbVideos', ".mp4")
        
## dropdown list to select video to play
videoSelected = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)
display(videoSelected)
#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Button
videoButton = widgets.Button(
    description='Play video'
)
video_output = widgets.Output()
def explore_video(b):
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="80%" height="80%" controls><source src="./data/rgbVideos/{fileName}" type="video/mp4"></video>'.format(fileName = videoSelected.value)
        video_output.append_display_data(HTML(html_video_code))


videoButton.on_click(explore_video)

clearButton = widgets.Button(
    description='Clear Playback'
)

def clear_video(b):
    with video_output:
        video_output.clear_output()
        
# display(clearButton)
clearButton.on_click(clear_video)

## to align the buttons to 1 row
display(clearButton)
display(videoButton, video_output)

This Folder contains 1 file(s).


Dropdown(description='Video: ', options=('P02T01C06.mp4',), value='P02T01C06.mp4')

Button(description='Clear Playback', style=ButtonStyle())

Button(description='Play video', style=ButtonStyle())

Output()

# 2. Inference Section

## Load a pretrained model

In [10]:
model_dir = './TSU/models/'
# listdir() returns a list containing the names of the entries in the directory given by path.
modelList = os.listdir("./TSU/models")

selectModel = widgets.Dropdown(
    options = modelList,
    value = modelList[0],
    description = "Choose a pre-trained model",
    disabled= False
)

selectModel
# print(selectModel.value)

Dropdown(description='Choose a pre-trained model', options=('PDAN_TSU_RGB', 'README.md'), value='PDAN_TSU_RGB'…

## Choose an input video from the TSU project

In [11]:
import os

## retrieve all TSU videos to dropdown list
folder_files = os.listdir('data/rgbVideos') 
print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
fileList=[]
for file in folder_files:
    fileList.append(file)

selected_input_video = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)
display(selected_input_video)

select_button = widgets.Button(
    description='Select video'
)
display(select_button)


# dropdown selected value --> selected_input_video.value
# def get_selected_value(b):
#     print(selected_input_video.value)
# select_button.on_click(get_selected_value)

This Folder contains 1 file(s).


Dropdown(description='Video: ', options=('P02T01C06.mp4',), value='P02T01C06.mp4')

Button(description='Select video', style=ButtonStyle())

## Inference Results

In [21]:
##Install dependencies

!pip install -r requirements_forTSU.txt

In [ ]:
#see inference results in the form of output video with captions indicating detected activity in each video frame

pretrainModel = selectModel.value
videoName = selected_input_video.value
dir = "./TSU/models"+pretrainModel

%run ./TSU/test.py -load_model dir
#%run ./TSU/test.py -load_model ./TSU/models/PDAN_TSU_RGB


# 3. Training Section

## Choose a dataset or upload your own

In [15]:
# upload own dataset from computer
def upload_dataset(file_val):
    if not file_val:
        print ("No file uploaded")

    else:   
        clear_output(wait=True)
        uploaded_file = next(iter(file_val))
        uploaded_filename = uploaded_file["name"]
        content = uploaded_file["content"]
        print("File name: " + uploaded_filename)

        try:
            save_path = './data/dataset'
            completeName = os.path.join(save_path, uploaded_filename) 
            with open(completeName, 'wb') as f: f.write(content)
            print (uploaded_filename + " uploaded to 'datasets' successfully!")
            
            # resets
            uploaded_file = ()
            uploaded_filename = ""
            content = ""
        except:
            print(sys.exc_info())

def show_upload(b):
    label_3 = widgets.Label("Upload your own dataset file below:")

    clear_output(wait=True)
    uploader = widgets.FileUpload(accept='.json',  # Currently only accepts .json
                                  multiple=False,
                                  description = "Browse",
                                  _counter = 0
    )
    upload_button = widgets.Button(
        description='Upload dataset',
        disabled = False
    )
    
    def on_button_clicked(b):
        upload_dataset(uploader.value)
        
    upload_button.on_click(on_button_clicked)
    display(label_3, uploader, upload_button)            
            
# select from pre-existing list of datasets (under ./datasets directory)
datasetList = populateList("./data/dataset", ".json") ## directory to be changed dependending how josh saves it as 

if not datasetList:
    print ("No existing datasets found")

selectedDataset = widgets.Dropdown(
    options = datasetList,
    value= datasetList[0],
    description="Dataset: ",
    disabled=False,
)

label_1 = widgets.Label("Select a pre-existing dataset")
label_2 = widgets.Label("or upload your own")

selectown_button = widgets.Button(
    description='Select own',
    disabled = False
)
selectown_button.on_click(show_upload)

# ui formatting
existingBox = widgets.VBox([label_1, selectedDataset])
newBox = widgets.VBox([label_2, selectown_button])
selectBox = widgets.HBox([existingBox, newBox])
display(selectBox)


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/dataset'

## initialize model

In [ ]:
# initialize model with a network architecture configured in a separate .py file
#"code here..."


## Set model

In [16]:
# Specify a name for this new model using appropriate UI elements.
model_name = widgets.Text(
    placeholder='Type something..',
    description='Model name:',
    disabled=False
)

confirm_button = widgets.Button(
    description='Confirm',
    disabled = False
)

def on_button_clicked(b):
    clear_output(wait=True)
    print("Textbox value is: "+ model_name.value)

confirm_button.on_click(on_button_clicked)
inputBox = widgets.HBox([model_name, confirm_button])
display(inputBox)

## Set the batch_size and epochs

In [17]:
# Set the batch_size and epochs, with appropriate UI elements, prior to running a training sequence
# Specify a name for this new model using appropriate UI elements.

label_setParams = widgets.Label("========================== Set parameters ==========================")

# to re set min/max values 
batch_size = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='Batch size:',
    disabled=False
)
epochs = widgets.BoundedIntText(
    min=1,
    # max=1000, # to be defined and added as necessary
    step=1,
    description='Epoch:',
    disabled=False
)

confirm_button2 = widgets.Button(
    description='Confirm',
    disabled = False
)

def on_button_clicked(b):
    print("batch_size value is: "+ str(batch_size.value))
    print("epochs value is: "+ str(epochs.value))

confirm_button2.on_click(on_button_clicked)
inputBox = widgets.VBox([label_setParams, batch_size, epochs, confirm_button2])
display(inputBox)

## Run the training sequence, i.e., fit the model onto the dataset

In [18]:
# Run the training sequence, i.e., fit the model onto the dataset
split_setting = ""
if "CS" in selectedDataset.value:
    split_setting = "CS"
elif "CV" in selectedDataset.value:
    split_setting = "CV"
    
print(split_setting, batchSize.value, epoch.value, modelName.value, selectedDataset.value)

## currently wont be able to run, need edit the train.py to allow new args "model_name" to save the model as this name
%run ./train.py -split_setting split_setting -batch_size batchSize -epoch epoch.value -model_name modelName.value

NameError: name 'selectedDataset' is not defined

## Visuals for progress of training

In [ ]:
# See some visual elements to indicate the progress of the training in the notebook

## After training is complete, the trained model should be added to the list of pre-trained models that can be chosen in R3

In [ ]:
# After training is complete, the trained model should be added to the list of pre-trained models that can be chosen in R3

# 4. Testing Section

## Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

In [ ]:
# Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

# replicated from section 3
existingBox = widgets.VBox([label_1, selectedDataset])
newBox = widgets.VBox([label_2, selectown_button])
selectBox = widgets.HBox([existingBox, newBox])
display(selectBox)


## Load a pretrained model using an appropriate UI component

In [ ]:
# Load a pretrained model using an appropriate UI component

# replicated from section 2
selectModel
# print(selectModel.value)

## Run the testing sequence

In [ ]:
# Run the testing sequence, i.e., perform inference on each data sample and accumulate some statistics
# (again check out the train.py file)

## See some visual elements to indicate the progress of testing in the notebook

In [ ]:
# •	View some results 
# (e.g., Average Precision per activity class and mean Average Precision) 
# that allow for an assessment of how well the model performed

## Save the results to a results folder in the repo

In [ ]:
# Save the results to a results folder in the repo

# 5. New Pipeline (NVIDIA STEP)

In [ ]:
## STEP is just one other HOI project we suggest, but you are free to use any others. A good resource may be https://paperswithcode.com